#### Align ephys events to a master clock

**Choose session to analyze**

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import os
from open_ephys.analysis import Session
import pandas as pd

In [ ]:
input_root_dir = Path('/ceph/sjones/projects/FlexiVexi/behavioural_data')
OUTPUT = Path("/ceph/sjones/projects/FlexiVexi/Data Analysis/intermediate_variables")


# (Example session with both TTL and heartbeat events):
animal_ID = 'FNT103'
session_ID = '2024-08-26T14-37-42'
#animal_ID = 'FNT104'
#session_ID = '2024-06-12T09-37-41'

**Load recording**
* Load recording object
* Load continuous.dat from stream 'NI-DAQmx-103.PXIe-6341'

In [ ]:

def get_record_node_path(root_folder):
    # Traverse the directory tree
    for dirpath, dirnames, filenames in os.walk(root_folder):
        # Check if 'settings.xml' is in the current directory
        if 'settings.xml' in filenames:
            return dirpath
        else:
            print('No recording found')

def get_session_path(root_folder):
    # Traverse the directory tree
    folder_one_level_up = None
    for dirpath, dirnames, filenames in os.walk(root_folder):
        # Check if any file ends with 'settings.xml'
        for filename in filenames:
            if filename.endswith('settings.xml'):
                # Get the folder one level up
                folder_one_level_up = os.path.dirname(dirpath)
                return folder_one_level_up
    if folder_one_level_up is None:
        print('No recording found')



# Find the path to the recording session
session_folder = input_root_dir / animal_ID / session_ID
print(session_folder)
ephys_session_path = get_session_path(session_folder)
print(ephys_session_path)

# load recording
session = Session(ephys_session_path)
recording = session.recordnodes[0].recordings[0]

# Get pandas data frame of continuous.dat on stream 'NI-DAQmx-103.PXIe-6341'
continuous_data = recording.continuous[1].get_samples(start_sample_index=0, end_sample_index=40000)
continuous_data = pd.DataFrame(continuous_data)
continuous_data

Each session will have a series of record nodes. For us, just one:

In [ ]:
print(session)

Each recording object has the following fields:

continuous : continuous data for each subprocessor in the recording  
spikes : spikes for each electrode group  
events : Pandas DataFrame of event times and metadata  

In [ ]:
# Get pandas data frame of continuous.dat on stream 'NI-DAQmx-103.PXIe-6341'
continuous_data = recording.continuous[1].get_samples(start_sample_index=0, end_sample_index=40000)
print(continuous_data)
continuous_data = pd.DataFrame(continuous_data)
continuous_data

Continuous data for each recording is accessed via the .continuous property of each Recording object. This returns a list of continuous data, grouped by processor/sub-processor. For example, if you have two data streams merged into a single Record Node, each data stream will be associated with a different processor ID. If you're recording Neuropixels data, each probe's data stream will be stored in a separate sub-processor, which must be loaded individually.

Each continuous object has four properties:

samples - a numpy.ndarray that holds the actual continuous data with dimensions of samples x channels. For Binary, NWB, and Kwik format, this will be a memory-mapped array (i.e., the data will only be loaded into memory when specific samples are accessed). 

sample_numbers - a numpy.ndarray that holds the sample numbers since the start of acquisition. This will have the same size as the first dimension of the samples array 

timestamps - a numpy.ndarray that holds global timestamps (in seconds) for each sample, assuming all data streams were synchronized in this recording. 
This will have the same size as the first dimension of the samples array  

metadata - a dict containing information about this data, such as the ID of the processor it originated from.  

##  Loading events

## Loading event data

Event data for each recording is accessed via the `.events` property of each `Recording` object. This returns a pandas DataFrame with the following columns:

- `sample_number` - the sample index at which this event occurred
- `timestamps` - the global timestamp (in seconds) at which this event occurred (defaults to -1 if all streams were not synchronized)
- `channel` - the channel on which this event occurred
- `processor_id` - the ID of the processor from which this event originated
- `stream_index` - the index of the stream from which this event originated
- `state` - 1 or 0, to indicate whether this is a rising edge or falling edge event. 

What is a `line` here? We have 1 and 4 of those,and they both look like they oscillate as a fixed square wave with different periods. 

In [ ]:
recording.events

In [ ]:
line_4 = recording.events[(recording.events["line"]==4) & (recording.events['processor_id']==103)]

In [ ]:
line_4

In [ ]:
beg =  1
end = 10
plt.plot(line_4['timestamp'][beg:end],line_4['state'][beg:end])

In [ ]:
# check whether events exist on line 4
events = recording.events

# get unique elements of stream_name
data_streams = list(set(events['stream_name']))

print(len(data_streams))
print(data_streams)

In [ ]:
event_df = recording.events


TTL_pulses = event_df[(event_df['stream_name'] == 'PXIe-6341') & (event_df['line'] == 4)]
TTL_pulses = TTL_pulses.reset_index(drop=True)
TTL_pulses

fig, ax = plt.subplots()
ax.plot(TTL_pulses['timestamp'], TTL_pulses['state'])
ax.set_xlim(0, 100)
ax.set_xlabel('Timestamp (s)')
ax.set_ylabel('TTL in PXIe board')
fig.suptitle('')

In [ ]:
TTL_pulses

In [ ]:
# check whether sync line can be added

# Sync line corresponding to heartbeat signal of ephys clock (1 pulse per second of duration 0.5 seconds). 
# Use this as the master clock (set main = True).
recording.add_sync_line(1,                          # 'Heartbeat' signal line number
                        100,                        # processor ID
                        'ProbeA',                   # stream name
                        main=True)                  # use as the main stream


# Sync line corresponding to TTL pulses
recording.add_sync_line(1,                          # TTL line number
                        102,                        # processor ID
                        'PXIe-6341',                # stream name
                        main=False)                 # synchronize to main stream

#THE PROCESSOR ID IS 102, NOT 103??

In [ ]:
recording.compute_global_timestamps(overwrite=False)

In [ ]:
event_df = recording.events
event_df

In [ ]:
TTL_pulses = event_df[(event_df['stream_name'] == 'PXIe-6341') & (event_df['line'] == 4)]
TTL_pulses = TTL_pulses.reset_index(drop=True)
TTL_pulses

# save the TTL pulses to a csv file
#TTL_pulses.to_csv('TTL_pulses.csv', index=False)

In [ ]:
TTL_pulses

In [ ]:


plt.plot(TTL_pulses['global_timestamp'], TTL_pulses['state'])
plt.xlim(0, 100)

## Implement and verify alignment

In [ ]:
from timestamps.harp.get_harp_timestamps_df import harp_session
from timestamps.OpenEphys.open_ephys_utils import openephys_session

animal_ID = 'FNT103'
session_ID = '2024-08-26T14-37-42'
#animal_ID = 'FNT104'
#session_ID = '2024-06-12T09-37-41'

harp = harp_session(animal_ID, session_ID)
harp.read_ttl()
harp.plot_ttl(100)

oe = openephys_session(animal_ID, session_ID)
oe.read_TTLs()
oe.plot_TTLs()
oe.sync_harp_ttls()

harp.ttl_state_df['global_timestamp'] = oe.tm.get_pxie_timestamp(harp.ttl_state_df['timestamp'])


In [ ]:
inc = 0

In [ ]:
import matplotlib.pyplot as plt

fig, ax =  plt.subplots()
ax.plot(oe.TTL_pulses['global_timestamp'], oe.TTL_pulses['state'])
harp.ttl_state_df['global_timestamp'] = oe.tm.get_pxie_timestamp(harp.ttl_state_df['timestamp'])
ax.plot(harp.ttl_state_df['global_timestamp'], harp.ttl_state_df['state'])
ax.set_xlim((0+inc, 100+inc))
fig.suptitle(f'{0+inc} to {100+inc}')
inc +=  100